<a href="https://colab.research.google.com/github/RinayGajjar/Customer_Churn_Prediction/blob/main/ANN_Churning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install tensorflow
import tensorflow as tf
print(tf.__version__)

2.17.1


In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [36]:
df = pd.read_csv('Churn_Modelling.csv')

In [37]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis = 1)

In [39]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [40]:
##encoding categorical
label_gender = LabelEncoder()
df['Gender'] = label_gender.fit_transform(df['Gender'])

In [41]:
df['Gender']

,Gender
0,0
1,0
2,0
3,0
4,0
...,...
9995,1
9996,1
9997,0
9998,1


In [42]:
##onehot encode geography
from sklearn.preprocessing import OneHotEncoder
OneHot_Encoder = OneHotEncoder()
df_geography = (OneHot_Encoder.fit_transform(df[['Geography']]))
OneHot_Encoder

OneHotEncoder()

In [43]:
OneHot_Encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [44]:
df_geography.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [45]:
geo_encoded_df = pd.DataFrame(df_geography.toarray(),columns = OneHot_Encoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [46]:
from re import A
##combine encoded with og data
df = pd.concat([df.drop(['Geography'],axis = 1),geo_encoded_df],axis = 1)

In [47]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [48]:
##dividing into independent and dependent variables
X= df.drop(['Exited'],axis = 1)
y = df['Exited']

In [49]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [50]:
y

,Exited
0,1
1,0
2,1
3,0
4,0
...,...
9995,0
9996,0
9997,1
9998,1


In [51]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [52]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential ##for creating neural nets
from tensorflow.keras.layers import Dense ## for creating hidden layers nodes
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

In [57]:
model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),##HL1
    Dense(32, activation='relu'),##HL2
    Dense(1,activation='sigmoid')##output layer (since binary classification so used sigmoid)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [70]:
##setting up tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
log_directory = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_directory,histogram_freq = 1,profile_batch = 100000000)

In [63]:
##setting up early stopping for training only for certain epochs
early_stopping_callback = EarlyStopping(monitor = 'val_loss',patience = 5, restore_best_weights = True)

In [64]:
##training out NN
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),
    epochs = 100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7650 - loss: 0.5076 - val_accuracy: 0.8290 - val_loss: 0.3923
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8393 - loss: 0.3927 - val_accuracy: 0.8515 - val_loss: 0.3593
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8457 - loss: 0.3686 - val_accuracy: 0.8565 - val_loss: 0.3472
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8596 - loss: 0.3431 - val_accuracy: 0.8570 - val_loss: 0.3447
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8630 - loss: 0.3331 - val_accuracy: 0.8555 - val_loss: 0.3450
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8654 - loss: 0.3272 - val_accuracy: 0.8580 - val_loss: 0.3460
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8681 - loss: 0.3272 - val_accuracy: 0.8615 - val_loss: 0.3419
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8693 - loss: 0.3172 - val_acc

In [73]:
##%load_ext tensorboard
##%tensorboard --logdir logs/fit/
%reload_ext tensorboard

In [78]:
from logging import makeLogRecord
input_data = {
    'CreditScore': 600
    , 'Age': 40
    , 'Tenure': 3
    , 'Balance': 60000
    , 'NumOfProducts': 2
    , 'HasCrCard': 1
    , 'IsActiveMember': 1,
    'EstimatedSalary':50000,
    'Gender':'Male',
    'Geography':'France'
}


In [82]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography
0,600,40,3,60000,2,1,1,50000,Male,France


In [79]:
geo_encoded_df = OneHot_Encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded_df,columns = OneHot_Encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [83]:
input_df['Gender'] = label_gender.transform([input_data['Gender']])
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography
0,600,40,3,60000,2,1,1,50000,1,France


In [84]:
input_df = pd.concat([input_df.drop(['Geography'],axis = 1),geo_encoded_df],axis = 1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,1,1.0,0.0,0.0


In [87]:
# Get the column names the scaler was trained on
training_columns = X.columns

# Ensure input_df has the same columns and order
input_df = input_df.reindex(columns=training_columns)

# Now you can scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [88]:
##prediction
pred = model.predict(input_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


array([[0.03066625]], dtype=float32)

In [90]:
pred_proba = pred[0][0]
pred_proba

0.030666253

In [92]:
if pred_proba > 0.5:
  print("Customer will Churn")
else:
  print("Customer will not Churn")

Customer will not Churn
